<a href="https://colab.research.google.com/github/HWatanuki/Projeto_D3TOP/blob/main/Data_preprocessing_v0.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>Instituto Federal de Educação, Ciência e Tecnologia
>>Câmpus Campinas<br>
>>D3TOP – Tópicos em Ciência de Dados<br>
>>Prof.: Samuel Botter Martins<br>

## Preprocessing and data clean step

### Environment Config

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [3]:
%%capture
!pip install -U spacy
!pip install -U gensim
!pip install -U neattext
!pip install -U pyspellchecker
!pip install -U stanza 
!pip install unidecode
!python -m spacy download pt_core_news_lg

In [4]:
# Importando bibliotecas e setando parâmetros

import numpy as np
import pandas as pd
from IPython.display import display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#### Useful Functions

In [5]:
def remove_cols(df, mask_cmds, inverse=True):
  '''
  Remove colunas de um dado dataframe df, a partir de uma máscara booleana mask.

  Parameters:
    df (pd.Dataframe): dataframe no qual sofrera a operacao de remocao de colunas
    mask_cmds (list): array de expressões a serem avaliadas. As expressões deve retornar array de booleanos contendo True nas posições das colunas que se deseja remover
    inverse (bool): (opcional) - indica se a máscara será invertida ou não. Default: True

  Returns:
    df (pd.Dataframe): dataframe sem as colunas indicadas na máscara
  '''
  mask_exp = mask_cmds.pop(0)
  mask = eval(mask_exp)
  if inverse:
    return remove_cols(df[df.columns[~mask]], mask_cmds) if len(mask_cmds) else df[df.columns[~mask]]
  return remove_cols(df[df.columns[mask]], mask_cmds) if len(mask_cmds) else df[df.columns[mask]]

In [6]:
def missing_values_table(df):
  '''
  Funcao que retorna o calculo de valores ausentes, absoluto e percentual, das colunas de um dado dataframe pandas

  Parameters:
    df (pd.Dataframe): dataframe que será  avaliado

  Returns:
    df (pd.Dataframe): dataframe contendo nas linhas o nome das colunas do dataframe original, e nas colunas
    a quantidade total de valores ausentes e o seu percentual correspondente.
  '''
  # Total missing values
  mis_val = df.isnull().sum()
  
  # Percentage of missing values
  mis_val_percent = 100 * df.isnull().sum() / len(df)
  
  # Make a table with the results
  mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
  
  # Rename the columns
  mis_val_table_ren_columns = mis_val_table.rename(
  columns = {0 : 'Missing Values', 1 : '% of Total Values'})
  
  # Sort the table by percentage of missing descending
  mis_val_table_ren_columns = mis_val_table_ren_columns[
      mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
  '% of Total Values', ascending=False).round(1)
  
  # Print some summary information
  print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
      "There are " + str(mis_val_table_ren_columns.shape[0]) +
        " columns that have missing values.")
  
  # Return the dataframe with missing information
  return mis_val_table_ren_columns

In [7]:
def neat_preprocessing(text_in: str, ) -> str:
  import neattext.functions as ntx

  text = text_in.lower()
  
  # text = ntx.fix_contractions(text)
  text = ntx.remove_html_tags(text)
  text = ntx.remove_punctuations(text) 
  text = ntx.replace_term(text, ntx.USER_HANDLES_REGEX, 'USUARIO')  
  text = ntx.remove_urls(text)
  text = ntx.remove_emails(text)
  text = ntx.remove_phone_numbers(text)
  text = ntx.remove_numbers(text)  
  text = ntx.replace_emojis(text, "EMOJI")
  text = ntx.remove_multiple_spaces(text)
  text = ntx.remove_currency_symbols(text)
  # text = ntx.remove_special_characters(text)
  
  return text


### Data gathering

The data gathering procedure was done through [Apify](https://apify.com/dtrungtin/airbnb-scraper/input-schema#locationQuery), a web scraping and automation platform.

In [8]:
%cd /content/drive/MyDrive/Colab Notebooks/IFSP/Trabalhos_Grupo_IFSP/NLP

/content/drive/MyDrive/Colab Notebooks/IFSP/Trabalhos_Grupo_IFSP/NLP


In [9]:
file_path = 'data/raw/'
file_name = 'dataset_airbnb-scraper_2023-04-04_01-45-49-997.csv'

df_raw = pd.read_csv(file_path + file_name, dtype=str)
print(f'No Rows: {df_raw.shape[0]}')
print(f'No Columns: {df_raw.shape[1]}', end='\n\n')
display(df_raw.head())

No Rows: 547
No Columns: 1107



,additionalHosts/0/about,additionalHosts/0/badges/0,additionalHosts/0/firstName,additionalHosts/0/hasInclusionBadge,additionalHosts/0/hasProfilePic,additionalHosts/0/hostUrl,additionalHosts/0/id,additionalHosts/0/isSuperHost,additionalHosts/0/languages/0,additionalHosts/0/languages/1,additionalHosts/0/languages/2,additionalHosts/0/languages/3,additionalHosts/0/languages/4,additionalHosts/0/memberSince,additionalHosts/0/pictureUrl,additionalHosts/0/responseRate,additionalHosts/0/responseTime,additionalHosts/0/smartName,additionalHosts/0/thumbnailUrl,additionalHosts/1/about,additionalHosts/1/badges/0,additionalHosts/1/firstName,additionalHosts/1/hasInclusionBadge,additionalHosts/1/hasProfilePic,additionalHosts/1/hostUrl,additionalHosts/1/id,additionalHosts/1/isSuperHost,additionalHosts/1/languages/0,additionalHosts/1/languages/1,additionalHosts/1/memberSince,additionalHosts/1/pictureUrl,additionalHosts/1/responseRate,additionalHosts/1/responseTime,additionalHosts/1/smartName,additionalHosts/1/thumbnailUrl,additionalHosts/2/about,additionalHosts/2/firstName,additionalHosts/2/hasInclusionBadge,additionalHosts/2/hasProfilePic,additionalHosts/2/hostUrl,additionalHosts/2/id,additionalHosts/2/isSuperHost,additionalHosts/2/memberSince,additionalHosts/2/pictureUrl,additionalHosts/2/smartName,additionalHosts/2/thumbnailUrl,address,isAvailable,isHostedBySuperhost,location/lat,location/lng,name,numberOfGuests,photos/0/caption,photos/0/pictureUrl,photos/0/thumbnailUrl,photos/1/caption,photos/1/pictureUrl,photos/1/thumbnailUrl,photos/2/caption,photos/2/pictureUrl,photos/2/thumbnailUrl,photos/3/caption,photos/3/pictureUrl,photos/3/thumbnailUrl,photos/4/caption,photos/4/pictureUrl,photos/4/thumbnailUrl,photos/5/caption,photos/5/pictureUrl,photos/5/thumbnailUrl,photos/6/caption,photos/6/pictureUrl,photos/6/thumbnailUrl,photos/7/caption,photos/7/pictureUrl,photos/7/thumbnailUrl,photos/8/caption,photos/8/pictureUrl,photos/8/thumbnailUrl,photos/9/caption,photos/9/pictureUrl,photos/9/thumbnailUrl,photos/10/caption,photos/10/pictureUrl,photos/10/thumbnailUrl,photos/11/caption,photos/11/pictureUrl,photos/11/thumbnailUrl,photos/12/caption,photos/12/pictureUrl,photos/12/thumbnailUrl,photos/13/caption,photos/13/pictureUrl,photos/13/thumbnailUrl,photos/14/caption,photos/14/pictureUrl,photos/14/thumbnailUrl,photos/15/caption,photos/15/pictureUrl,photos/15/thumbnailUrl,photos/16/caption,photos/16/pictureUrl,photos/16/thumbnailUrl,photos/17/caption,photos/17/pictureUrl,photos/17/thumbnailUrl,photos/18/caption,photos/18/pictureUrl,photos/18/thumbnailUrl,photos/19/caption,photos/19/pictureUrl,photos/19/thumbnailUrl,photos/20/caption,photos/20/pictureUrl,photos/20/thumbnailUrl,photos/21/caption,photos/21/pictureUrl,photos/21/thumbnailUrl,photos/22/caption,photos/22/pictureUrl,photos/22/thumbnailUrl,photos/23/caption,photos/23/pictureUrl,photos/23/thumbnailUrl,photos/24/caption,photos/24/pictureUrl,photos/24/thumbnailUrl,photos/25/caption,photos/25/pictureUrl,photos/25/thumbnailUrl,photos/26/caption,photos/26/pictureUrl,photos/26/thumbnailUrl,photos/27/caption,photos/27/pictureUrl,photos/27/thumbnailUrl,photos/28/caption,photos/28/pictureUrl,photos/28/thumbnailUrl,photos/29/caption,photos/29/pictureUrl,photos/29/thumbnailUrl,photos/30/caption,photos/30/pictureUrl,photos/30/thumbnailUrl,photos/31/caption,photos/31/pictureUrl,photos/31/thumbnailUrl,photos/32/caption,photos/32/pictureUrl,photos/32/thumbnailUrl,photos/33/caption,photos/33/pictureUrl,photos/33/thumbnailUrl,photos/34/caption,photos/34/pictureUrl,photos/34/thumbnailUrl,photos/35/caption,photos/35/pictureUrl,photos/35/thumbnailUrl,photos/36/caption,photos/36/pictureUrl,photos/36/thumbnailUrl,photos/37/caption,photos/37/pictureUrl,photos/37/thumbnailUrl,photos/38/caption,photos/38/pictureUrl,photos/38/thumbnailUrl,photos/39/caption,photos/39/pictureUrl,photos/39/thumbnailUrl,photos/40/caption,photos/40/pictureUrl,photos/40/thumbnailUrl,photos/41/caption,photos/41/pictureUrl,photos/41/thumbnailUrl,photos

### Step 01 - discarding information that doesn't matter

Removing columns relating to:
- `additionalHosts`
- property `photos` (we'll keep just first) 
- URL's
- `primaryHost/languages` (we'll keep just first) 

In [10]:
mask_additionalHosts = "df.columns.str.startswith('additionalHosts')"
mask_photos = "df.columns.str.contains(r'^photos/[^0]')"
mask_url = "df.columns.str.contains(r'^(?!photos).*Url$')"
mask_phlang = "df.columns.str.contains(r'^primaryHost/languages/[^0]')"
masks_commands = [mask_additionalHosts, mask_photos, mask_url, mask_phlang]

df_s01 = df_raw.pipe(remove_cols, masks_commands)

print(f'No Columns before step 01: {df_raw.shape[1]}')
print(f'No Columns after step 01: {df_s01.shape[1]}', end='\n\n')
df_s01.sample(1)

No Columns before step 01: 1107
No Columns after step 01: 452



,address,isAvailable,isHostedBySuperhost,location/lat,location/lng,name,numberOfGuests,photos/0/caption,photos/0/pictureUrl,photos/0/thumbnailUrl,pricing/rate/amount,pricing/rate/amountFormatted,pricing/rate/currency,pricing/rate/isMicrosAccuracy,pricing/rateType,primaryHost/about,primaryHost/badges/0,primaryHost/badges/1,primaryHost/badges/2,primaryHost/firstName,primaryHost/hasInclusionBadge,primaryHost/hasProfilePic,primaryHost/id,primaryHost/isSuperHost,primaryHost/languages/0,primaryHost/memberSince,primaryHost/responseRate,primaryHost/responseTime,primaryHost/smartName,reviews/0/author/firstName,reviews/0/author/hasProfilePic,reviews/0/author/id,reviews/0/author/smartName,reviews/0/collectionTag,reviews/0/comments,reviews/0/createdAt,reviews/0/id,reviews/0/language,reviews/0/localizedDate,reviews/0/localizedReview,reviews/0/localizedReview/comments,reviews/0/localizedReview/disclaimer,reviews/0/localizedReview/needsTranslation,reviews/0/localizedReview/response,reviews/0/rating,reviews/0/recipient/firstName,reviews/0/recipient/hasProfilePic,reviews/0/recipient/id,reviews/0/recipient/smartName,reviews/0/response,reviews/1/author/firstName,reviews/1/author/hasProfilePic,reviews/1/author/id,reviews/1/author/smartName,reviews/1/collectionTag,reviews/1/comments,reviews/1/createdAt,reviews/1/id,reviews/1/language,reviews/1/localizedDate,reviews/1/localizedReview,reviews/1/localizedReview/comments,reviews/1/localizedReview/disclaimer,reviews/1/localizedReview/needsTranslation,reviews/1/localizedReview/response,reviews/1/rating,reviews/1/recipient/firstName,reviews/1/recipient/hasProfilePic,reviews/1/recipient/id,reviews/1/recipient/smartName,reviews/1/response,reviews/2/author/firstName,reviews/2/author/hasProfilePic,reviews/2/author/id,reviews/2/author/smartName,reviews/2/collectionTag,reviews/2/comments,reviews/2/createdAt,reviews/2/id,reviews/2/language,reviews/2/localizedDate,reviews/2/localizedReview,reviews/2/localizedReview/comments,reviews/2/localizedReview/disclaimer,reviews/2/localizedReview/needsTranslation,reviews/2/localizedReview/response,reviews/2/rating,reviews/2/recipient/firstName,reviews/2/recipient/hasProfilePic,reviews/2/recipient/id,reviews/2/recipient/smartName,reviews/2/response,reviews/3/author/firstName,reviews/3/author/hasProfilePic,reviews/3/author/id,reviews/3/author/smartName,reviews/3/collectionTag,reviews/3/comments,reviews/3/createdAt,reviews/3/id,reviews/3/language,reviews/3/localizedDate,reviews/3/localizedReview,reviews/3/localizedReview/comments,reviews/3/localizedReview/disclaimer,reviews/3/localizedReview/needsTranslation,reviews/3/localizedReview/response,reviews/3/rating,reviews/3/recipient/firstName,reviews/3/recipient/hasProfilePic,reviews/3/recipient/id,reviews/3/recipient/smartName,reviews/3/response,reviews/4/author/firstName,reviews/4/author/hasProfilePic,reviews/4/author/id,reviews/4/author/smartName,reviews/4/collectionTag,reviews/4/comments,reviews/4/createdAt,reviews/4/id,reviews/4/language,reviews/4/localizedDate,reviews/4/localizedReview,reviews/4/localizedReview/comments,reviews/4/localizedReview/disclaimer,reviews/4/localizedReview/needsTranslation,reviews/4/localizedReview/response,reviews/4/rating,reviews/4/recipient/firstName,reviews/4/recipient/hasProfilePic,reviews/4/recipient/id,reviews/4/recipient/smartName,reviews/4/response,reviews/5/author/firstName,reviews/5/author/hasProfilePic,reviews/5/author/id,reviews/5/author/smartName,reviews/5/collectionTag,reviews/5/comments,reviews/5/createdAt,reviews/5/id,reviews/5/language,reviews/5/localizedDate,reviews/5/localizedReview,reviews/5/localizedReview/comments,reviews/5/localizedReview/disclaimer,reviews/5/localizedReview/needsTranslation,reviews/5/localizedReview/response,reviews/5/rating,reviews/5/recipient/firstName,reviews/5/recipient/hasProfilePic,reviews/5/recipient/id,reviews/5/recipient/smartName,reviews/5/response,reviews/6/author/firstName,reviews/6/author/hasProfilePic,reviews/6/author/id,reviews/6/author/smart

### Step 02 - Evaluate Missing Values

In [11]:
missing_values_table(df_s01)

Your selected dataframe has 452 columns.
There are 429 columns that have missing values.


,Missing Values,% of Total Values
reviews/2/collectionTag,547,100.000
reviews/15/collectionTag,547,100.000
reviews/9/localizedReview,547,100.000
reviews/1/localizedReview,547,100.000
reviews/9/collectionTag,547,100.000
reviews/14/collectionTag,547,100.000
reviews/8/localizedReview,547,100.000
reviews/18/localizedReview/response,547,100.000
reviews/14/localizedReview,547,100.000
reviews/18/localizedReview,547,100.000


### Step 03 - discarding information based on missing values

keeping only the following columns about reviews:
- Review: `comments`, `createdAt`, `id`, `rating`
- Review Author: `firstName`, `id`

In [12]:
mask_reviews_data = "df.columns.str.contains(r'^reviews/[0-9]+/(?!author/)(?!comments|createdAt|id|rating)')"
mask_author_data = "df.columns.str.contains(r'^reviews/[0-9]+/author/(?!firstName|id)')"
masks_commands = [mask_reviews_data, mask_author_data]

df_s03 = df_s01.pipe(remove_cols, masks_commands)

print(f'No Columns before step 02: {df_s01.shape[1]}')
print(f'No Columns after step 02: {df_s03.shape[1]}', end='\n\n')
df_s03.sample(1)

No Columns before step 02: 452
No Columns after step 02: 152



,address,isAvailable,isHostedBySuperhost,location/lat,location/lng,name,numberOfGuests,photos/0/caption,photos/0/pictureUrl,photos/0/thumbnailUrl,pricing/rate/amount,pricing/rate/amountFormatted,pricing/rate/currency,pricing/rate/isMicrosAccuracy,pricing/rateType,primaryHost/about,primaryHost/badges/0,primaryHost/badges/1,primaryHost/badges/2,primaryHost/firstName,primaryHost/hasInclusionBadge,primaryHost/hasProfilePic,primaryHost/id,primaryHost/isSuperHost,primaryHost/languages/0,primaryHost/memberSince,primaryHost/responseRate,primaryHost/responseTime,primaryHost/smartName,reviews/0/author/firstName,reviews/0/author/id,reviews/0/comments,reviews/0/createdAt,reviews/0/id,reviews/0/rating,reviews/1/author/firstName,reviews/1/author/id,reviews/1/comments,reviews/1/createdAt,reviews/1/id,reviews/1/rating,reviews/2/author/firstName,reviews/2/author/id,reviews/2/comments,reviews/2/createdAt,reviews/2/id,reviews/2/rating,reviews/3/author/firstName,reviews/3/author/id,reviews/3/comments,reviews/3/createdAt,reviews/3/id,reviews/3/rating,reviews/4/author/firstName,reviews/4/author/id,reviews/4/comments,reviews/4/createdAt,reviews/4/id,reviews/4/rating,reviews/5/author/firstName,reviews/5/author/id,reviews/5/comments,reviews/5/createdAt,reviews/5/id,reviews/5/rating,reviews/6/author/firstName,reviews/6/author/id,reviews/6/comments,reviews/6/createdAt,reviews/6/id,reviews/6/rating,reviews/7/author/firstName,reviews/7/author/id,reviews/7/comments,reviews/7/createdAt,reviews/7/id,reviews/7/rating,reviews/8/author/firstName,reviews/8/author/id,reviews/8/comments,reviews/8/createdAt,reviews/8/id,reviews/8/rating,reviews/9/author/firstName,reviews/9/author/id,reviews/9/comments,reviews/9/createdAt,reviews/9/id,reviews/9/rating,reviews/10/author/firstName,reviews/10/author/id,reviews/10/comments,reviews/10/createdAt,reviews/10/id,reviews/10/rating,reviews/11/author/firstName,reviews/11/author/id,reviews/11/comments,reviews/11/createdAt,reviews/11/id,reviews/11/rating,reviews/12/author/firstName,reviews/12/author/id,reviews/12/comments,reviews/12/createdAt,reviews/12/id,reviews/12/rating,reviews/13/author/firstName,reviews/13/author/id,reviews/13/comments,reviews/13/createdAt,reviews/13/id,reviews/13/rating,reviews/14/author/firstName,reviews/14/author/id,reviews/14/comments,reviews/14/createdAt,reviews/14/id,reviews/14/rating,reviews/15/author/firstName,reviews/15/author/id,reviews/15/comments,reviews/15/createdAt,reviews/15/id,reviews/15/rating,reviews/16/author/firstName,reviews/16/author/id,reviews/16/comments,reviews/16/createdAt,reviews/16/id,reviews/16/rating,reviews/17/author/firstName,reviews/17/author/id,reviews/17/comments,reviews/17/createdAt,reviews/17/id,reviews/17/rating,reviews/18/author/firstName,reviews/18/author/id,reviews/18/comments,reviews/18/createdAt,reviews/18/id,reviews/18/rating,reviews/19/author/firstName,reviews/19/author/id,reviews/19/comments,reviews/19/createdAt,reviews/19/id,reviews/19/rating,roomType,stars,url
484,"Vila Luis Antonio, São Paulo, Brasil",true,false,-24.00498,-46.26745,Visão do Paraíso na Praia das Astúrias,10,Vista praias Pitangueiras_Enseada,https://a0.muscache.com/im/pictures/miso/Hosting-51361278/original/c4fd017a-6685-4ad5-b18c-1103885e0be7.jpeg?aki_policy=large,https://a0.muscache.com/im/pictures/miso/Hosting-51361278/original/c4fd017a-6685-4ad5-b18c-1103885e0be7.jpeg?aki_policy=small,1186,R$1.186,BRL,false,nightly,NaN,26 avaliações,Identidade verificada,NaN,Kátia,false,true,42947957,false,NaN,Membro desde agosto de 2015,100%,dentro de algumas horas,Kátia,Andréia,494635545,"O check-in foi um pouco demorado pois quando cheguei na recepção meu nome é dos outros convidados não constavam,o porteiro ,muito gentil ,disse saber que a cobertura estava alugada porém sem a ordem da proprietária não poderia liberar,mais ou menos 20 minutos esse contratempo foi resolvido.<br/>Ao entrar ficamos realmente encantados com a cobertura,sua vista a organização da casa ,tudo impecável até melhor que as fotos <

### Step 04 - Getting text columns from dataframe

In [13]:
# Definindo as  colunas de texto
mask_text_cols = "df.columns.str.contains(r'name|about|comments')"
df_txt_cols = df_s03.pipe(remove_cols, [mask_text_cols], inverse=False)
df_txt_cols.sample(1)

,name,primaryHost/about,reviews/0/comments,reviews/1/comments,reviews/2/comments,reviews/3/comments,reviews/4/comments,reviews/5/comments,reviews/6/comments,reviews/7/comments,reviews/8/comments,reviews/9/comments,reviews/10/comments,reviews/11/comments,reviews/12/comments,reviews/13/comments,reviews/14/comments,reviews/15/comments,reviews/16/comments,reviews/17/comments,reviews/18/comments,reviews/19/comments
374,Mansão a beira mar em Residencial Marina Guarujá,NaN,"A casa impecável, muito bem cuidada, muito bem decorada, quartos e banheiros imensos, ainda mais bonita pessoalmente, sem contar a área de piscina com a vista para o canal, sensacional, realmente uma bela de uma mansão.<br/>Já a Sra Helena é um caso a parte, ela e sua filha muito educadas, sempre dispostas a ajudar, sem contar o maravilhoso café da manha e almoço que faziam para nós.<br/>Resumindo foi muito melhor do que eu imaginava.",A casa é simplesmente Maravilhosa!<br/>A Funcionária Helena foi essencial para tornar nossa estádia excepcional! Voltarei sem sombra de dúvidas com toda minha família!!<br/>Giuliana/Bruno vcs estão de parabéns pela casa que superou todas as nossas expectativas!!,"Foi tudo perfeito! Giuliana é um doce, estava sempre de prontidão para tirar minhas dúvidas e ainda fez uma vídeo chamada pra me mostrar a casa antes de me hospedar. O condomínio é lindo e a casa mais ainda, um luxo! Ela possui um pier onde as lanchas podem fazer parada pra pegar o pessoal que quer fazer passeios, nossa família amou demaisss! Foi tudo lindo, um final de semana inesquecível! Dona Helena que cuida da casa é um amor também, a calma em pessoa! Esperamos voltar em breve!!!",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
arr_texts = df_txt_cols.fillna('').values
arr_texts[:5]

array([['Ap 2q conforto e comodidade!', '',
        'The host canceled this reservation 107 days before arrival. This is an automated posting.',
        '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', ''],
       ['Apartamento Guarujá Praia Enseada Sp (GRUPO REIS)', '', '', '',
        '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', ''],
       ['Casa na Praia de Pernambuco, Guarujá - SP', '',
        'A casa fica bem próxima da praia, além disso a casa é uma graça, tudo muito limpo e organizado. Os anfritriões são muito atenciosos. Recomendo, voltaremos mais vezes!',
        'Um lugar lindo e próximo a praia, da para ir de a pé.  Cristian e dona Maria super atenciosos e cuidadosos conosco. Casa super limpa e com utensílios que dá para cozinhar sem problemas. A praia é de água limpa e bem familiar. Amamos o lugar e pretendemos voltar mais vezes.  Valeu muito a pena !',
        'Cristian e Dona Maria excelentes anfitriões! <br/

### Step 05 - Defining preprocessing and cleaning operations

In [15]:
from functools import cache
from typing import Union
import spacy
spacy.load("pt_core_news_lg")
import stanza
stanza.download('pt')

@cache
def preprocess_txt(textraw:str, nlp_obj:Union[spacy.lang.pt.Portuguese, stanza.pipeline.core.Pipeline], lemma=False) -> str:
  '''
  Realiza a limpeza e o pre processamento de um dado texto em portugues.
  As etapas de pré-processamento a serem executadas são:
  * Conversão para Minúsculas.
  * Remocao de tags HTML.
  * Remocao de pontuacoes.
  * Remocao de algarismos numericos e numeros de telefone.
  * Remocao de multiplos espacos.
  * Remocao de URLs e E-mails: são removidos, pois eles não adicionam significado ao texto.
  * Substituicao de Emojis: Substitui emojis usando um dicionário predefinido contendo emojis junto com seu significado. (por exemplo: ":)" para "sorriso")
  * Substituindo nomes de usuário: Substitua @Usernames pela palavra "USUARIO". (por exemplo: "@nick_name" para "USUARIO")
  * Remocao de ascentuacao.
  * Removendo Não-Alfabetos: Substitui caracteres (exceto Dígitos e Alfabetos) por um espaço.
  * Removendo letras consecutivas: 3 ou mais letras consecutivas são substituídas por 2 letras. (por exemplo: "Muitoooo" para "Muitoo")
  * Remoção de palavras curtas: palavras com comprimento menor que 2 são removidas.
  * Removendo Stopwords: Stopwords são as palavras que não adicionam muito significado a uma frase. Eles podem ser ignorados com segurança sem sacrificar o significado da frase. (por exemplo: "the", "he", "have")
  * Lematização: A lematização é o processo de converter uma palavra em sua forma básica. (por exemplo: “Great” para “Good”). A lematização geralmente retorna palavras válidas (que existem) enquanto as técnicas de stemming retornam (na maioria das vezes) palavras abreviadas

  Parameters:
    textraw (str): texto bruto o qual sofrera as operacoes de limpeza e pre processamento.
    nlp_obj (spacy.lang.pt.Portuguese | stanza.pipeline.core.Pipeline): - objeto que indica a biblioteca a ser utilizada para a etapa de tokenizacao, remocao de stop words e lemmatizacao.
    lemma (boolean): (opcional) se deve substituir as palavras originais do texto por seu lemma ou nao. Default: False

  Returns:
    wordsclean (pd.Dataframe): dataframe sem as colunas indicadas na máscara
  '''
  import re
  from unidecode import unidecode
  # from gensim.utils import simple_preprocess
  
    

  # Definindo padrões Regex
  # urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
  # userPattern       = '@[^\s]+'
  alphaPattern      = "[^a-zA-Z0-9]"
  sequencePattern   = r"(.)\1\1+" # letras consecutivas
  seqReplacePattern = r"\1\1"
  
  # Definindo dicionario de emojis e seus respectivos significados
  emojis = {':)': 'sorri', ':-)': 'sorri', ';d': 'pisca', ':-E': 'vampiro', ':(': 'triste', 
            ':-(': 'triste', ':-<': 'triste', ':P': 'mostra lingua', ':O': 'surpresa',
            ':-@': 'chocado', ':@': 'chocado',':-$': 'confuso', ':\\': 'irritado', 
            ':#': 'mudo', ':X': 'mudo', ':^)': 'sorri', ':-&': 'confuso', '$_$': 'ganancia',
            '@@': 'vira os olhos', ':-!': 'confuso', ':-D': 'sorri', ':-0': 'grita', 'O.o': 'confuso',
            '<(-_-)>': 'robo', 'd[-_-]b': 'dj', ":'-)": 'sorri triste', ';)': 'piscar', 
            ';-)': 'piscar', 'O:-)': 'anjo','O*-)': 'anjo','(:-D': 'fofoca', '=^.^=': 'gato'}

   
  # ##################
  # Tentativa de aplicação de técnica para correção de palavras com erro de escrita, 
  # mas o resultado não foi considerado satisfatório.
  # ##################
  # from gensim.utils import simple_tokenize
  # from spellchecker import SpellChecker
  # spell = SpellChecker(language='pt')
  # wordlist = list(simple_tokenize(textraw))
  # misspelled = spell.unknown(wordlist)
  # correctedtext = [spell.correction(word) for word in misspelled]
  # print(correctedtext)

  textclean = neat_preprocessing(textraw)  
  for emoji in emojis.keys():
      textclean = textclean.replace(emoji, "EMOJI" + emojis[emoji])  
  textclean = re.sub(sequencePattern, seqReplacePattern, textclean)
  textclean = unidecode(textclean)
  textclean = re.sub(alphaPattern, " ", textclean)
  wordsclean = ''

  if isinstance(nlp_obj, spacy.lang.pt.Portuguese):
    doc = nlp_obj(textclean)
    for token in doc:      
      if not token.is_stop and len(token) > 1:
        if lemma and token.pos_ == 'VERB':
          wordsclean += token.lemma_+' '
        else:
          wordsclean += token.text + ' '

  elif isinstance(nlp_obj, stanza.pipeline.core.Pipeline):
    doc = nlp_obj(textclean)
    for sent in doc.sentences:
      for word in sent.words:      
        if lemma and word.upos == 'VERB':
          wordsclean += word.lemma +' '
        else:
          wordsclean += word.text + ' '
  else:
      raise BaseException('Instancia nao reconhecida do objeto "nlp_obj"')
    
  return wordsclean


INFO:stanza:Downloading default packages for language: pt (Portuguese) ...
INFO:stanza:File exists: /root/stanza_resources/pt/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


### Step 06 - Tests

In [16]:
arr_texts[204]

array(['COBERTURA PANORÂMICA GUARUJÁ PITANGUEIRAS COM SPA', '',
       'Nossa estadia foi maravilhosa , belíssima vista da praia , sem falar na Cida ótima cozinheira , nota 1000 para Risoto !',
       'Adoramos tudo e a Cida uma querida 🤩🙏🏻',
       'O apartamento está bem localizado  , possui uma vista para Mar privilegiada. O atendimento do Flavio e da Cida foi impecável. Super indico',
       'Flávio e Cida foram muito gentis. O apartamento é realmente bom e fica em frente a praia. É lindo! A Jacuzzi não estava aquecendo, porém o proprietário disse estar regularizando.',
       'Apartamento muito bom, aconchegante, ótima localização. A funcionária Cida é maravilhosa, atenciosa e cozinha extremamente bem, ficamos muito satisfeitos.',
       'Flávio muito atencioso e Cida quem nos ajuda no período da estadia é formidável.',
       'Eu e minha família amamos o apartamento do sr. Flávio. Localização maravilhosa, vista linda e com a jacuzzi, churrasqueira e mesa de bilhar não precisamos 

#### Teste Spacy

In [17]:
%%time
nlp_spacy = spacy.load("pt_core_news_lg", disable=['parser', 'ner', 'textcat'])
postprocess = []
for rawtxt in arr_texts[204]:
  if len(rawtxt):
    postprocess.append(preprocess_txt(rawtxt, nlp_obj=nlp_spacy, lemma=True))

postprocess

CPU times: user 1.09 s, sys: 325 ms, total: 1.41 s
Wall time: 1.43 s


['cobertura panoramica guaruja pitangueiras spa ',
 'estadia maravilhosa belissima ver praia falar cida otima cozinheira nota risoto ',
 'adorar cida querida EMOJI ',
 'apartamento localizar possuir vista mar privilegiada atendimento flavio cida impecavel super indico ',
 'flavio cida gentis apartamento realmente ficar frente praia lir jacuzzi nao aquecer porem proprietario dizer regularizar ',
 'apartamento aconchegante otima localizacao funcionaria cida maravilhosa atenciosa cozinha extremamente ficar satisfeitos ',
 'flavio atencioso cida ajuda periodo estadia formidavel ',
 'familia amamos apartamento sr flavio localizacao maravilhosa vista linda jacuzzi churrasqueira mesa bilhar nao precisar sair apartamento divertir cida empregada sr flavio educada pontual cozinha divertido voltar futuro ',
 'perfeita cobertura localizacao espetacular conforme planejar ver espetacular melhor pessoalmente opiniao melhor fotos airbnb estao parabens cuidado receptividade cida recomendar assunto amig

####  Teste Stanza

In [18]:
%%time
########## Stanza defined's processors for this task:
# tokenize - Segments a Document into Sentences, each containing a list of Tokens.
# mwt - Expands multi-word tokens (MWTs) into multiple words when they are predicted by the tokenizer.Fix contractions.
# pos - UPOS, XPOS, and UFeats annotations are accessible through Word’s properties `pos`, `xpos`, and `ufeats`.
# lemma - Perform lemmatization on a Word using the `Word.text` and `Word.upos` values. The result can be accessed as `Word.lemma`. 
nlp_stanza = stanza.Pipeline(lang='pt', processors='tokenize,mwt,pos,lemma', use_gpu=True, download_method=None, verbose=False, max_cache_size=2)
postprocess = []
for rawtxt in arr_texts[204]:
  if len(rawtxt):
    postprocess.append(preprocess_txt(rawtxt, nlp_obj=nlp_stanza, lemma=True))

postprocess

CPU times: user 2.69 s, sys: 946 ms, total: 3.64 s
Wall time: 3.71 s


['cobertura panoramica guaruja pitangueiras com spa ',
 'nossa estadia foi maravilhosa belissima ver de a praia sem falar em a cida otima cozinheira nota para risoto ',
 'adorar tudo e a cida uma querida EMOJI ',
 'o apartamento esta bem localizar possuir uma vista para mar privilegiar o atendimento de o flavio e de a cida foi impecavel super indicar ',
 'flavio e cida foram muito gentis o apartamento e realmente bom e ficar em frente a praia e lir a jacuzzi em ao estava aquecer porem o proprietario dizer estar regularizar ',
 'apartamento muito bom aconchegante otima localizacao a funcionar cida e maravilhosa atenciosa e cozinha extremamente bem ficar muito satisfeitos ',
 'flavio muito atencioso e cida quem nos ajudar em o periodo de a estadia e formidavel ',
 'eu e minha familia amar o apartamento de o sr flavio localizacao maravilhosa ver linda e com a jacuzzi churrasqueira e mesa de bilhar em ao precisar sair de o apartamento para nos divertir a cida empregar de o sr flavio e muit

### Step 06 - Applying preprocessing operations on dataframe text columns

In [22]:
%%time
df_txt_cols_processed = df_txt_cols.applymap(preprocess_txt, na_action='ignore', nlp_obj=nlp_spacy, lemma=True)

CPU times: user 25.9 s, sys: 5.24 ms, total: 25.9 s
Wall time: 26 s


In [24]:
df_txt_cols_processed.sample(5)

,name,primaryHost/about,reviews/0/comments,reviews/1/comments,reviews/2/comments,reviews/3/comments,reviews/4/comments,reviews/5/comments,reviews/6/comments,reviews/7/comments,reviews/8/comments,reviews/9/comments,reviews/10/comments,reviews/11/comments,reviews/12/comments,reviews/13/comments,reviews/14/comments,reviews/15/comments,reviews/16/comments,reviews/17/comments,reviews/18/comments,reviews/19/comments
336,mini wedding,morar casa moderna colocar casa alugar eventoscow workingfotosdia noiva,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,casa pra copa mundo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,ape dormcom garagem seguranca,NaN,condominio seguro muitas comodidades farmacia mercado restaurantes etc anfitria co anfitria sao atensiosas cuidadosas apartamento limpo adequado,localizacao farmacia mercado padaria metro,estadia voltar coanfritia rita atenciosa solicitar,apartamento perfeito localizar atendimento tranquilo recomer,excelente hospedarlimposeguro recepcao anfitria ritamaravilhosaEMOJImuito cordial educada,otima estadia Belem apartamento conforme anuncio otima recepcao recomendar,adorar apartamento super limpo organizar toalhas boas roupa cama condominio super seguro ter vaga garagem otimo recomender,guta otima anfitria apartamento impecavel completobem arejado confortavelrecomer,excelentes anfitrioes super atenciosa espaco limpo organizar nota defeitos estadia,apartamento excelente familiar proximo fizemos refeicoes completo perfeito criancas grade desenhos incrivel excelente custo beneficio sentir tao casa desejo morar apartamento voltaremos super recomer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
244,hotel vicino al mare premium frente mar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,lir frente berrini cama casal,NaN,tati proporcionar privacidade rapida responder,tatiane excelente anfitria localizacao apartamento excelenteobrigar tati retornar,localizacao apto otima limpeza modo poder melhorar,cama confortavel mesa trabalho localizacao otima padaria frente alem host super flexivel checkin extremamente comunicavel durante estadia,localizacao melhor possivel lugares precisar vila olimpia padaria super tradicional esquina refeicoes comunicacao facil acomodacoes adequadas limpas confortaveis,excelente estadia limpeza facilidade comunicacao prestativa checkin localizacao excepcional alimentacao servicos supermercados pex valer,tatiane dar certo dormi voce ofereceao acordar oportunidade tomar cafe manha luxo padoca brique ficar distancia minutos pe,tatiane atender super imprevisto acomodacao tati prontificar receber agradavel excelente padaria hrs frente tati dar dicas restaurantes tbm pena fiquei otimo,estadia excelente treinamento sheraton wtc iniciar terminar fatores check in check out tatiane excelente atenciosa momentos solicitar localizacao otima limpo bonito arejar,estadia esperar dormir tatiane super atenciosa conseguir garagem predio preco justo banheiro ok banho cama ok noite sono localizacao impecavel esquina berrini padaria hs frente predio opcao imaginar gostamos recomendar,otima localizacao limpeza impecavel querido voltar reservar hospedagem ajudar acidente compreensivos recomendo,excelente anfitrioes sao simpaticos faceis comunicar,estadia maravilhosa tatiane super simpatica,adoreer atenciosa recomendo,anfitrioes simpaticos otima localizacao proxima estacao berrini otima vista sp,otimo espaco recomendar,otimo super privativo,NaN,NaN,NaN


In [19]:
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm

tqdm.pandas()

def parallel_applymap(df, func, worker_count, **kwargs):
    def _apply(shard):
        return shard.progress_applymap(func, **kwargs)

    shards = np.array_split(df, worker_count)
    with ThreadPoolExecutor(max_workers=worker_count) as e:
        futures = e.map(_apply, shards)
    return pd.concat(list(futures))

In [20]:
%%time
df_txt_cols_processed = parallel_applymap(df_txt_cols, preprocess_txt, worker_count=10, na_action='ignore', nlp_obj=nlp_stanza, lemma=True)

  0%|          | 0/1210 [00:00<?, ?it/s]

  0%|          | 0/1210 [00:00<?, ?it/s]

  0%|          | 0/1210 [00:00<?, ?it/s]

  0%|          | 0/1210 [00:00<?, ?it/s]

  0%|          | 0/1210 [00:00<?, ?it/s]

...

  0%|          | 0/1210 [00:00<?, ?it/s]

  0%|          | 0/1188 [00:00<?, ?it/s]

  0%|          | 0/1188 [00:00<?, ?it/s]

  0%|          | 0/1188 [00:00<?, ?it/s]

CPU times: user 8min 3s, sys: 1min 16s, total: 9min 20s
Wall time: 6min 43s


In [21]:
df_txt_cols_processed.loc[133:134]

,name,primaryHost/about,reviews/0/comments,reviews/1/comments,reviews/2/comments,reviews/3/comments,reviews/4/comments,reviews/5/comments,reviews/6/comments,reviews/7/comments,reviews/8/comments,reviews/9/comments,reviews/10/comments,reviews/11/comments,reviews/12/comments,reviews/13/comments,reviews/14/comments,reviews/15/comments,reviews/16/comments,reviews/17/comments,reviews/18/comments,reviews/19/comments
133,casa de veraneio em a praia de pernambuco,NaN,a casa e perfeita para reunir a familia com seguranca e sossego sao cinco suites que oferecer privacidade para todos os hospedes reunir familiares em um ambiente gostor com uma piscina maravilhosa e limpa inclusive com som ambiente o ar condicionar e muito eficiente em todos os quartos a edna cozinha divinamente e o mauro deixar as dependencias imacular ela nos fazer uma lula a dore de os deuses super crocantes a caipirinha de o mauro e digno de um barmanrecomendo sem restricoes,mesmo sem conhecer pessoalmente a carolina ela sempre foi muito educada simpatica e rapida eficiente em as respostas a casa superar todas as nossas expectativas o local e enorme e acomodar confortavelmente as pessoas que viajar comigo os caseiros ficar todo o tempo em a casa e nos ajudar com algumas tarefas de o dia a dia com destaque para o feijao de a edna e o churrasco de o mauro esta em os meus planos voltar em breve para passar mais alguns dias,carolina s home was incredible the house and property are even more beautiful in person than in the pictures the house and pool were perfect for our multigenerational family and art lovers there were multiple lounge game and play areas the beach was just a few blocks away and many other stunning beaches to visit were a short drive away understanding how to work with the caseros was a bit of a learning curve because we did not speak portugues and were not accustomed to having them but after a day or two we figured it out and mauro edna were fantastic one thing that could be improved is the wifi did not extend throughout the entire house would highly recommend carolina s property and hope to visit again,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,alto padrao jardins,engenheiro empresario casado filhos,gil s apartment is fantastic location is great the apartment is spacious and cozy gil was really helpful and kind with us i recommend this place,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
